In [ ]:
import pandas as pd
import numpy as np

data = pd.read_parquet('../data/albania_res.parquet')

In [ ]:
data.index = pd.to_datetime(data.index, utc='UTC')

In [ ]:
from mlpforecast.data.transform import DatasetObjective
from sklearn.preprocessing import MinMaxScaler, RobustScaler

config_dict = {
    "target_series": "NetLoad",
    "unknown_features": ['NetLoad_rolling_mean_win_48',
       'NetLoad_rolling_mean_win_336'],
    "calender_variable": ['HOUR', 'Session'],
    "known_calender_features": ['HOUR-cosin', 'Session-cosin'],
    "known_continuous_features": ['NetLoad_lag_48', 'NetLoad_lag_336', 'Temperature', 'Humidity', 'Rain'],
    "input_scaler": RobustScaler(),
    "target_scaler":  RobustScaler(),
    "lags": [1 , 7 ],
    "window": [1 , 7 ],
    "window_func": ["mean"],
    "period": "30min",
    "input_window_size": 96,
    "forecast_horizon": 48,
    "date_column": "timestamp"
}

In [ ]:
ds= DatasetObjective(**config_dict)

In [ ]:
#data.reset_index().columns

In [ ]:
x, y=ds.fit_transform(data.reset_index())

In [ ]:
import arviz as az
import matplotlib
import matplotlib.pyplot as plt
az.style.use(["science", 'arviz-doc', 'tableau-colorblind10'])
nice_fonts = {
        "savefig.bbox" : "tight",
        "savefig.pad_inches" : 0.05,
        "axes.labelsize": 8,
        "font.size": 8,
        "legend.fontsize": 8,
        "legend.frameon" :False
}
matplotlib.rcParams.update(nice_fonts)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf", "retina")  # For export

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4,2))
ax.plot(x[0, :, :5]);

In [ ]:
from mlpforecast.data.loader import TimeseriesDataModule

dl = TimeseriesDataModule(x, y)

In [ ]:
x, y=next(iter(dl.train_dataloader()))
fig, ax = plt.subplots(1,1, figsize=(4,2))
ax.plot(x[0, :, :5]);

In [ ]:
from mlpforecast.forecaster.mlp import MLPForecast
hparams = {"data_pipeline":ds, "target_series": "NetLoad",
    "unknown_features": ['NetLoad_rolling_mean_win_48',
       'NetLoad_rolling_mean_win_336'],
    "known_calender_features": ['HOUR-cosin', 'Session-cosin'],
    "known_continuous_features": ['NetLoad_lag_48', 'NetLoad_lag_336', 'Temperature', 'Humidity', 'Rain'],
                "embedding_size": 20,
                "embedding_type": None,
                "combination_type": "additional",
                "hidden_size": 256,
                "num_layers": 2,
                'expansion_factor':2,
                "residual":  False,
                "forecast_horizon": 48,
                "input_window_size": 96,
                "activation_function": "SiLU",
                "out_activation_function": "Identity",
                "dropout_rate": 0.1,
                "alpha": 0.25,
                "max_epochs":10,
            }

model = MLPForecast(hparams=hparams)

In [ ]:
model.fit(data.reset_index()[:20000])

In [ ]:
x,y=next(iter(model.datamodule.train_dataloader()))

In [ ]:
model.model.eval()
pred=model.model.forecast(x)['pred']
fig, ax = plt.subplots(1,1, figsize=(4,2))
ax.plot(pred[0, :, 0]);
ax.plot(y[0, :, 0]);